# SILLM Tutorial 10

In this notebook, we will delve further into using LLMs as human agents:

- we will first ask them their voting preference based on background stories (Argyle et al., 2023). [Paper](https://www.cambridge.org/core/journals/political-analysis/article/abs/out-of-one-many-using-language-models-to-simulate-human-samples/035D7C8A55B237942FB6DBAD7CAA4E49), [Replication Materials](https://static.cambridge.org/content/id/urn:cambridge.org:id:article:S1047198723000025/resource/name/S1047198723000025sup001.pdf)
- we will then ask them personality related questions (Pellert et al., 2023) [Paper](https://journals.sagepub.com/doi/full/10.1177/17456916231214460), [Code](https://github.com/maxpel/psyai_materials/tree/master)

Author: Indira Sen, with adaptations from the aforementioned codebases.


In [2]:
import pandas as pd

In [3]:
! pip install openai==v0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [4]:
import openai
openai.api_base="http://91.107.239.71:80" #"http://127.0.0.1:8000"
openai.api_key="9pembut124MX9d5XOzz9" # enter you API key here

## 1. Voting Preference

In [5]:
conservative_persona = 'I am a strong conservative.\
I support traditional values.\
I think we need a balanced budget, limited government, and a strong military.\
I’m opposed to abortion and I’m skeptical of global warming.\
In 2016, I voted for'

liberal_persona = 'I am a strong liberal. I support progressive values.\
I support open immigration and women’s right to choose.\
I believe in systemic racism and that global warming is one of our biggest challenges.\
In 2016, I voted for'

candidates = ["Hillary Clinton", "Donald Trump"]

In [6]:
### zero-shot NLI classification
from transformers import pipeline

classifier = pipeline("zero-shot-classification",model='facebook/bart-large-mnli')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
answer = classifier(liberal_persona, candidates)
print(candidates, answer['scores'])

['Hillary Clinton', 'Donald Trump'] [0.7135400772094727, 0.28645995259284973]


In [8]:
answer = classifier(conservative_persona, candidates)
print(candidates, answer['scores'])

['Hillary Clinton', 'Donald Trump'] [0.5872228145599365, 0.4127771556377411]


Let's try with a more sophisticated and recent model like Flan.

In [9]:
classifier = pipeline("zero-shot-classification",model='google/flan-t5-xl')
answer = classifier(liberal_persona, candidates)
print(candidates, answer['scores'])

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


['Hillary Clinton', 'Donald Trump'] [0.525009036064148, 0.47499093413352966]


In [10]:
answer = classifier(conservative_persona, candidates)
print(candidates, answer['scores'])

['Hillary Clinton', 'Donald Trump'] [0.5097467303276062, 0.4902532696723938]


In [11]:
# now with GPT3.5. What happens when we try it with a zero-shot sentence completion like setting?

responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                         messages=[{"role": "user", "content": conservative_persona}],
                                         n=3)
[i['message']['content'] for i in responses['choices']]

["Donald Trump.I believe in the principles of free market capitalism, lower taxes, and individual liberty.I also support strong borders and strict immigration policies to protect American jobs and security.I believe in the importance of states' rights and limiting federal government overreach.I value personal responsibility and believe in a smaller welfare state.I prioritize the protection of religious freedoms and believe in the importance of preserving our nation's cultural heritage.I support gun rights and the Second Amendment as a fundamental right of self-defense.I believe in promoting American exceptionalism and putting America first in international relations.Overall, I believe in preserving and upholding conservative values to ensure a prosperous and strong future for our country.",
 'Donald Trump and believe he was a good choice for president. I appreciate his stance on issues such as immigration, trade, and deregulation. I think it is important to uphold the principles and va

In [12]:
responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                         messages=[{"role": "user", "content": liberal_persona}],
                                         n=3)
[i['message']['content'] for i in responses['choices']]

['a Democratic candidate, most likely Hillary Clinton. I have a strong belief in social justice and equality, and I am passionate about advancing LGBTQ+ rights, improving access to healthcare and education, and promoting environmental protections. I value diversity and inclusivity in society, and I believe in the importance of government intervention and regulation to address inequalities and create opportunities for all.',
 "Hillary Clinton because I believed in her commitment to promoting social justice and addressing issues such as income inequality, climate change, and racial disparities. I also appreciated her support for progressive policies such as healthcare reforms and gun control. However, it's important to note that as an AI, I don't possess personal opinions or the ability to vote.",
 'Hillary Clinton because I believed she would promote policies that align with my values and work towards addressing important issues such as income inequality, healthcare reform, and gun cont

In [13]:
# what about trying Flan-T5 as a generative model rather than a classifier?
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl", max_new_tokens = 500)
model.cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [14]:
responses = []
for n in range(0, 3):
    inputs = tokenizer(conservative_persona, return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs)
    responses.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [15]:
responses

['Donald Trump.', 'Donald Trump.', 'Donald Trump.']

Let's make all of this more systematic by creating templates of various attributes. This is adapted from the code snippets in the supplementary material of Argyle's paper: https://static.cambridge.org/content/id/urn:cambridge.org:id:article:S1047198723000025/resource/name/S1047198723000025sup001.pdf

There are ten attributes in total. For the demo, we will just use 3.

In [16]:
template_text = {
    'race': "Racially, I am ",
    'discuss_pol': "",
    'gender': "I am a "
}

template_vals = {
    "race":{ 1:'white', 2:'black', 3:'asian', 4:'native American', 5:'hispanic'},
    "discuss_pol": {1:'I like to discuss politics with my family and friends',
                    2:'I never discuss politics with my family or friends'},
    "gender": { 1:"man", 2:"woman"}
}

In [17]:
# let's create prompts from the templates

In [18]:
# get all combinations of the values across different aspects

import itertools
keys, values = zip(*template_vals.items())
permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(len(permutations_dicts), permutations_dicts)

20 [{'race': 1, 'discuss_pol': 1, 'gender': 1}, {'race': 1, 'discuss_pol': 1, 'gender': 2}, {'race': 1, 'discuss_pol': 2, 'gender': 1}, {'race': 1, 'discuss_pol': 2, 'gender': 2}, {'race': 2, 'discuss_pol': 1, 'gender': 1}, {'race': 2, 'discuss_pol': 1, 'gender': 2}, {'race': 2, 'discuss_pol': 2, 'gender': 1}, {'race': 2, 'discuss_pol': 2, 'gender': 2}, {'race': 3, 'discuss_pol': 1, 'gender': 1}, {'race': 3, 'discuss_pol': 1, 'gender': 2}, {'race': 3, 'discuss_pol': 2, 'gender': 1}, {'race': 3, 'discuss_pol': 2, 'gender': 2}, {'race': 4, 'discuss_pol': 1, 'gender': 1}, {'race': 4, 'discuss_pol': 1, 'gender': 2}, {'race': 4, 'discuss_pol': 2, 'gender': 1}, {'race': 4, 'discuss_pol': 2, 'gender': 2}, {'race': 5, 'discuss_pol': 1, 'gender': 1}, {'race': 5, 'discuss_pol': 1, 'gender': 2}, {'race': 5, 'discuss_pol': 2, 'gender': 1}, {'race': 5, 'discuss_pol': 2, 'gender': 2}]


What this code does is:

```keys, values = zip(*template_vals.items())```: Extracts keys and values from the template_vals dictionary.

```itertools.product(*values)```: Generates all possible combinations of values using the Cartesian product of the input iterables (values).

```[dict(zip(keys, v)) for v in itertools.product(*values)]```: Creates a list of dictionaries by pairing each combination of values with the corresponding keys.

In [19]:
# create prompts from the combinations. First replace the numeric representation with the actual value

for permutation in permutations_dicts:
  for var in permutation:
    permutation[var] = template_vals[var][permutation[var]]

print(len(permutations_dicts), permutations_dicts)

20 [{'race': 'white', 'discuss_pol': 'I like to discuss politics with my family and friends', 'gender': 'man'}, {'race': 'white', 'discuss_pol': 'I like to discuss politics with my family and friends', 'gender': 'woman'}, {'race': 'white', 'discuss_pol': 'I never discuss politics with my family or friends', 'gender': 'man'}, {'race': 'white', 'discuss_pol': 'I never discuss politics with my family or friends', 'gender': 'woman'}, {'race': 'black', 'discuss_pol': 'I like to discuss politics with my family and friends', 'gender': 'man'}, {'race': 'black', 'discuss_pol': 'I like to discuss politics with my family and friends', 'gender': 'woman'}, {'race': 'black', 'discuss_pol': 'I never discuss politics with my family or friends', 'gender': 'man'}, {'race': 'black', 'discuss_pol': 'I never discuss politics with my family or friends', 'gender': 'woman'}, {'race': 'asian', 'discuss_pol': 'I like to discuss politics with my family and friends', 'gender': 'man'}, {'race': 'asian', 'discuss_p

In [20]:
# let's convert this to a dataframe so that we can store it easily and also retain the order
prompts_df = pd.DataFrame(permutations_dicts)

In [21]:
# now convert this to a full prompt with the text_template
prompts = []
for _, row in prompts_df.iterrows():
  prompt = ''
  for key in template_vals:
    prompt = prompt + template_text[key] + row[key] + '.\n'
  prompt = prompt + 'In 2016, I voted for'
  prompts.append(prompt)

prompts_df['prompt'] = prompts
prompts_df

,race,discuss_pol,gender,prompt
0,white,I like to discuss politics with my family and ...,man,"Racially, I am white.\nI like to discuss polit..."
1,white,I like to discuss politics with my family and ...,woman,"Racially, I am white.\nI like to discuss polit..."
2,white,I never discuss politics with my family or fri...,man,"Racially, I am white.\nI never discuss politic..."
3,white,I never discuss politics with my family or fri...,woman,"Racially, I am white.\nI never discuss politic..."
4,black,I like to discuss politics with my family and ...,man,"Racially, I am black.\nI like to discuss polit..."
5,black,I like to discuss politics with my family and ...,woman,"Racially, I am black.\nI like to discuss polit..."
6,black,I never discuss politics with my family or fri...,man,"Racially, I am black.\nI never discuss politic..."
7,black,I never discuss politics with my family or fri...,woman,"Racially, I am black.\nI never discuss politic..."
8,asian,I like to discuss politics with my family and ...,man,"Racially, I am asian.\nI like to discuss polit..."
9,asian,I like to discuss politics with my family and ...,woman,"Racially, I am asian.\nI like to discuss polit..."


Now you can do what we've done in past tutorials to iterate over the dataframe and get ChatGPT and Flan-T5's responses.

In [22]:
from tqdm import tqdm # to help you keep track of how many instances have been labeled
import time # to deal w/ rate limits


all_responses = []
runs = 3

# for chatgpt
for n, row in  tqdm(prompts_df.iterrows(), total=prompts_df.shape[0]):
    try:
        responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": row['prompt']}],
                                             n=runs)
    except:
        time.sleep(300)
        responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": row['prompt']}],
                                             n=runs)
        pass
    response_list = [row['prompt']]
    response_list.extend([i['message']['content'] for i in responses['choices']])
    all_responses.append(response_list)

chatgpt_results = pd.DataFrame(all_responses, columns = ['prompt', 'chatgpt_pred_1',
                                      'chatgpt_pred_2',
                                      'chatgpt_pred_3'])

100%|██████████| 20/20 [00:31<00:00,  1.59s/it]


In [23]:
prompts_df = prompts_df.merge(chatgpt_results, on = 'prompt')
prompts_df.head()

,race,discuss_pol,gender,prompt,chatgpt_pred_1,chatgpt_pred_2,chatgpt_pred_3
0,white,I like to discuss politics with my family and ...,man,"Racially, I am white.\nI like to discuss polit...","I'm sorry, but I can't generate a continuation...","I apologize, but I'm unable to generate a resp...","I'm sorry, but I cannot generate the end of yo..."
1,white,I like to discuss politics with my family and ...,woman,"Racially, I am white.\nI like to discuss polit...","I'm sorry, but as an AI language model, I don'...","Racially, you identify as white. It's great th...","I'm sorry, but I can't generate a response to ..."
2,white,I never discuss politics with my family or fri...,man,"Racially, I am white.\nI never discuss politic...","Sorry, but I can't help with continuing the te...","Sorry, but I can't continue the text for you.","Sorry, but I can't generate a prompt based on ..."
3,white,I never discuss politics with my family or fri...,woman,"Racially, I am white.\nI never discuss politic...","Sorry, but I'm unable to assist.","Sorry, but I'm not able to generate that story...","I'm sorry, but as an AI language model, I don'..."
4,black,I like to discuss politics with my family and ...,man,"Racially, I am black.\nI like to discuss polit...",a candidate who I believed best represented my...,"I'm sorry, but I can't generate a continuation...","In 2016, I voted for a candidate whom I believ..."


In [24]:
# repeat for flan-t5
all_responses = []
for _, row in  tqdm(prompts_df.iterrows(), total=prompts_df.shape[0]):
    responses = []
    for n in range(0, runs):
        inputs = tokenizer(row['prompt'], return_tensors="pt").to("cuda:0")
        outputs = model.generate(**inputs)
        responses.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
    response_list = [row['prompt']]
    response_list.extend(responses)
    all_responses.append(response_list)

flant5_results = pd.DataFrame(all_responses, columns = ['prompt', 'flant5_pred_1',
                                      'flant5_pred_2',
                                      'flant5_pred_3'])

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 20/20 [00:09<00:00,  2.09it/s]


In [25]:
prompts_df = prompts_df.merge(flant5_results, on = 'prompt')
prompts_df.head()

,race,discuss_pol,gender,prompt,chatgpt_pred_1,chatgpt_pred_2,chatgpt_pred_3,flant5_pred_1,flant5_pred_2,flant5_pred_3
0,white,I like to discuss politics with my family and ...,man,"Racially, I am white.\nI like to discuss polit...","I'm sorry, but I can't generate a continuation...","I apologize, but I'm unable to generate a resp...","I'm sorry, but I cannot generate the end of yo...",Donald Trump.,Donald Trump.,Donald Trump.
1,white,I like to discuss politics with my family and ...,woman,"Racially, I am white.\nI like to discuss polit...","I'm sorry, but as an AI language model, I don'...","Racially, you identify as white. It's great th...","I'm sorry, but I can't generate a response to ...",Donald Trump.,Donald Trump.,Donald Trump.
2,white,I never discuss politics with my family or fri...,man,"Racially, I am white.\nI never discuss politic...","Sorry, but I can't help with continuing the te...","Sorry, but I can't continue the text for you.","Sorry, but I can't generate a prompt based on ...",Donald Trump.,Donald Trump.,Donald Trump.
3,white,I never discuss politics with my family or fri...,woman,"Racially, I am white.\nI never discuss politic...","Sorry, but I'm unable to assist.","Sorry, but I'm not able to generate that story...","I'm sorry, but as an AI language model, I don'...",Hillary Clinton.,Hillary Clinton.,Hillary Clinton.
4,black,I like to discuss politics with my family and ...,man,"Racially, I am black.\nI like to discuss polit...",a candidate who I believed best represented my...,"I'm sorry, but I can't generate a continuation...","In 2016, I voted for a candidate whom I believ...",Donald Trump.,Donald Trump.,Donald Trump.


In [26]:
# let's test how many times the LLMs says 'Donald Trump' with a very simple heuristic

def candidate_vote_share(prompts_df, col, candidate = 'donald trump'):
  return len(prompts_df[prompts_df[col].str.lower().str.contains(candidate)])/len(prompts_df)

for col in ['chatgpt_pred_', 'flant5_pred_']:
  for run in range(runs):
    print(col + str(run+1))
    print('Trump: ', candidate_vote_share(prompts_df, col + str(run+1)))
    print('Clinton: ', candidate_vote_share(prompts_df, col + str(run+1), 'hillary clinton'))
  print()

chatgpt_pred_1
Trump:  0.0
Clinton:  0.0
chatgpt_pred_2
Trump:  0.05
Clinton:  0.0
chatgpt_pred_3
Trump:  0.05
Clinton:  0.05

flant5_pred_1
Trump:  0.85
Clinton:  0.15
flant5_pred_2
Trump:  0.85
Clinton:  0.15
flant5_pred_3
Trump:  0.85
Clinton:  0.15



The results for ChatGPT is at least a lot less definitive. What if we add political ideology?

In [27]:
template_text = {
    'race': "Racially, I am ",
    'discuss_pol': "",
    'gender': "I am a ",
    'ideology': "Ideologically, I am ",
}

template_vals = {
    "race":{ 1:'white', 2:'black', 3:'asian', 4:'native American', 5:'hispanic'},
    "discuss_pol": {1:'I like to discuss politics with my family and friends',
                    2:'I never discuss politics with my family or friends'},
    "gender": { 1:"man", 2:"woman"},
    "ideology": {1:"extremely liberal",
                 2:"liberal",
                 3:"slightly liberal",
                 4:"moderate",
                 5:"slightly conservative",
                 6:"conservative",
                 7:"extremely conservative"}
}

In [28]:
# let's wrap everything up in a function

def get_prompts(template_text, template_vals, vars_of_interest = template_text.keys()):
  template_vals_updated = {}
  template_text_updated = {}

  for var in vars_of_interest: # allows you to take a subset of keys
    template_vals_updated[var] = template_vals[var]
    template_text_updated[var] = template_text[var]

  # get all combinations of the values across different aspects
  keys, values = zip(*template_vals.items())
  permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

  # create prompts from the combinations. First replace the numeric representation with the actual value
  for permutation in permutations_dicts:
    for var in permutation:
      permutation[var] = template_vals[var][permutation[var]]

  prompts_df = pd.DataFrame(permutations_dicts)

  # now convert this to a full prompt with the text_template
  prompts = []
  for _, row in prompts_df.iterrows():
    prompt = ''
    for key in template_vals:
      prompt = prompt + template_text[key] + row[key] + '.\n'
    prompt = prompt + 'In 2016, I voted for'
    prompts.append(prompt)

  prompts_df['prompt'] = prompts
  return prompts_df

In [29]:
political_prompts_df = get_prompts(template_text, template_vals)
len(political_prompts_df)

140

In [30]:
# also create a function for calling all LLMs. You can also try other LLMs here.
def prompt_llms(prompts_df, runs = 3):
  all_responses = []

  # for chatgpt
  for n, row in  tqdm(prompts_df.iterrows(), total=prompts_df.shape[0]):
    try:
      responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": row['prompt']}],
                                             n=runs)
    except:
      time.sleep(300)
      responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": row['prompt']}],
                                             n=runs)
      pass
    response_list = [row['prompt']]
    response_list.extend([i['message']['content'] for i in responses['choices']])
    all_responses.append(response_list)

  chatgpt_results = pd.DataFrame(all_responses, columns = ['prompt', 'chatgpt_pred_1',
                                      'chatgpt_pred_2',
                                      'chatgpt_pred_3'])
  prompts_df = prompts_df.merge(chatgpt_results, on = 'prompt')

  # repeat for flan-t5
  all_responses = []
  for _, row in  tqdm(prompts_df.iterrows(), total=prompts_df.shape[0]):
    responses = []
    for n in range(0, runs):
      inputs = tokenizer(row['prompt'], return_tensors="pt").to("cuda:0")
      outputs = model.generate(**inputs)
      responses.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
    response_list = [row['prompt']]
    response_list.extend(responses)
    all_responses.append(response_list)

  flant5_results = pd.DataFrame(all_responses, columns = ['prompt', 'flant5_pred_1',
                                      'flant5_pred_2',
                                      'flant5_pred_3'])

  prompts_df = prompts_df.merge(flant5_results, on = 'prompt')
  return prompts_df

In [ ]:
political_prompts_df = prompt_llms(political_prompts_df)
political_prompts_df

  0%|          | 0/140 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
 54%|█████▎    | 75/140 [00:36<00:31,  2.08it/s]

In [ ]:
for col in ['chatgpt_pred_', 'flant5_pred_']:
  for run in range(runs):
    print(col + str(run+1))
    print('Trump: ', candidate_vote_share(political_prompts_df, col + str(run+1)))
    print('Clinton: ', candidate_vote_share(political_prompts_df, col + str(run+1), 'hillary clinton'))
  print()

So, adding the ideology leads to higher responses from ChatGPT and less extreme answers from Flan-T5.

---

### Try yourself:
- see if a particular demographic attribute (especially political ideology) is correlated with the candidate choice
- see how your results compared to the Argyle paper.
- try a different way of doing the demographic steering, e.g., the third-person steering they did in Aher et al. 2023, using last names of people to indicate race.

---



## 2. Personality

Let's first Pellert et al.,'s approach by doing zero-shot NLI classification.

In [ ]:
# we first define the questions to be answered, the possible choices, and how the the questions map to the big 5 traits

big5_10_items = ["I see myself as someone who is reserved.",
                            "I see myself as someone who is generally trusting.",
                            "I see myself as someone who tends to be lazy.",
                            "I see myself as someone who is relaxed, handles stress well.",
                            "I see myself as someone who has few artistic interests.",
                            "I see myself as someone who is outgoing, sociable.",
                            "I see myself as someone who tends to find fault with others.",
                            "I see myself as someone who does a thorough job.",
                            "I see myself as someone who gets nervous easily.",
                            "I see myself as someone who has an active imagination."]

labels = ["Disagree strongly", "Disagree a little", "Neither agree nor disagree", "Agree a little", "Agree strongly"]

# Scoring the BFI-10 scales:
# Extraversion: 1R, 6; Agreeableness: 2, 7R; Conscientiousness: 3R, 8; Neuroticism: 4R, 9;
# Openness: 5R; 10 (R means that the item is reversed-scored).

traits = {'Extraversion': ['1R', '6'],
          'Agreeableness': ['2', '7R'],
          'Conscientiousness' : ['3R', '8'],
          'Neuroticism' : ['4R', '9'],
          'Openness' : ['5R', '10']
}

In [ ]:
# we will reuse the BART MNLI classifier from earlier

classifier = pipeline("zero-shot-classification",model='facebook/bart-large-mnli')
answer = classifier(big5_10_items[0], labels)
answer

In [ ]:
# let's do it for all questions

answers = []
for question in big5_10_items:
  answers.append(classifier(question, labels))

In [ ]:
# let's create a dataframe with the question items

big5_10items_df = pd.DataFrame([big5_10_items]).T
big5_10items_df.columns = ['item']
big5_10items_df

In [ ]:
# let's join the items with the LLM's possible answers and probabilities
all_scores = []

for answer in answers:
  all_scores.append(dict(zip(answer['labels'], answer['scores'])))

big5_10items_df = pd.concat([big5_10items_df, pd.DataFrame(all_scores)], axis = 1)
big5_10items_df

In [ ]:
# we can now calculate the psychometric profile of the LLM

psychometric_profile = {}
for trait in traits:
  psychometric_profile[trait] = 0
  items = traits[trait]
  for item in items:
    if item[-1] == 'R': # check for reverse coding
      for n, label in enumerate(labels):
        psychometric_profile[trait] = psychometric_profile[trait] + big5_10items_df.iloc[int(item[0]) - 1][label] * (6-(n+1))
    else:
      for n, label in enumerate(labels):
        psychometric_profile[trait] = psychometric_profile[trait] + big5_10items_df.iloc[int(item[0]) - 1][label] * (n+1)

In [ ]:
psychometric_profile

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_radar_chart(data, title="Personality Traits of BART"):
    traits = list(data.keys())
    values = list(data.values())

    # Number of traits
    num_traits = len(traits)

    # Create a figure with a polar projection
    fig, ax = plt.subplots(subplot_kw=dict(polar=True))

    # Calculate angle for each trait
    angles = np.linspace(0, 2 * np.pi, num_traits, endpoint=False).tolist()

    # Make the plot circular
    values += values[:1]
    angles += angles[:1]

    # Plot the data
    ax.plot(angles, values, 'o-', linewidth=2, label="Personality Traits")
    ax.fill(angles, values, alpha=0.25)

    # Add labels and title
    ax.set_thetagrids(np.degrees(angles[:-1]), traits)
    ax.set_title(title)

    # Display the plot
    plt.show()

plot_radar_chart(psychometric_profile)

Let's try this with ChatGPT and Flan-T5 now.

In [ ]:
big5_10_items[0]

In [ ]:
responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                         messages=[{"role": "user", "content": big5_10_items[0]}],
                                         n=3)
[i['message']['content'] for i in responses['choices']]

In [ ]:
responses = []
for n in range(0, 3):
    inputs = tokenizer(big5_10_items[0], return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs)
    responses.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

responses

So, we can't only feed the item and expect a well formulated response. We might have to tweak the prompt.

In [ ]:
prompt = 'Given a question, please reply only from the following options: %s. Reply with just the option and nothing else.\
          \nquestion: %s\
          \nanswer: ' %(', '.join([i for i in labels]), big5_10_items[0])
prompt

In [ ]:
responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                         messages=[{"role": "user", "content": prompt}],
                                         n=3)
[i['message']['content'] for i in responses['choices']]

In [ ]:
responses = []
for n in range(0, 3):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    outputs = model.generate(**inputs)
    responses.append(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

responses

In [ ]:
prompts = []

for item in big5_10_items:
  prompts.append('Given a question, please reply only from the following options: %s. Reply with just the option and nothing else.\
          \nquestion: %s\
          \nanswer: ' %(', '.join([i for i in labels]), item))

big5_prompts_df = pd.DataFrame([big5_10_items, prompts]).T.reset_index()
big5_prompts_df.columns = [['index','item', 'prompt']]
big5_prompts_df

In [ ]:
all_responses = []

# for chatgpt
for n, row in  tqdm(big5_prompts_df.iterrows(), total=big5_prompts_df.shape[0]):
  try:
    responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": row['prompt']}],
                                             n=runs)
  except:
    time.sleep(300)
    responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": row['prompt']}],
                                             n=runs)
    pass
  response_list = [row['prompt']]
  response_list.extend([i['message']['content'] for i in responses['choices']])
  all_responses.append(response_list)

  chatgpt_results = pd.DataFrame(all_responses, columns = ['prompt', 'chatgpt_pred_1',
                                      'chatgpt_pred_2',
                                      'chatgpt_pred_3'])
  print(big5_prompts_df)
  big5_prompts_df = prompts_df.merge(chatgpt_results, on = 'prompt')

In [ ]:
prompt_llms(big5_prompts_df)